In [ ]:
from Config import *
from Data import *
from Model import *
from Metricas import *
import tensorflow as tf
import pandas as pd
import numpy as np

# Prueba 1 U-Ignore

In [ ]:

#Creación de los dataSet  con enfoque U-Ignore
df_train_ignore = crear_dataFrame_UIgnore('/train.csv', 0.5)
train_dataset_ignore = obtener_dataset_Aug(df_train_ignore)

df_valid_ignore = crear_dataFrame_UIgnore('/valid.csv', 1.0)
valid_dataset_ignore = obtener_dataset_Aug(df_valid_ignore)

In [ ]:
model_J = crear_modelo_J()

In [ ]:
#Entra Modelo Jorge U-Ignore sin pesos por clase
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    #Corta el entrenamiento automáticamente si el modelo deja de mejorar.
    EarlyStopping(monitor='val_auc', patience=4, mode='max', restore_best_weights=True),
    #Reduce automáticamente el learning rate si la métrica no mejora.
    #ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, mode='min')
    ReduceLROnPlateau(
    monitor='val_auc',     # Métrica a observar
    factor=0.5,            # Cuánto se reduce el learning rate (a la mitad en este caso)
    patience=2,            # Cuántas epochs esperar SIN mejora antes de reducir el LR
    mode='max',            # Porque queremos que el AUC crezca
    min_lr=1e-6,           # No baja más allá de este valor, máxima resolución del learing rate
    verbose=1              # Muestra en consola cuando se reduce
)
]
#Compilación:
model_J.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss= re_loss_P1, metrics=[tf.keras.metrics.AUC(name='auc')])
model_J.fit(train_dataset_ignore, epochs=9, validation_data= valid_dataset_ignore, callbacks = callbacks)

## Métricas
Métricas para modelo_J con data Augmentation enfoque U-ignore, 40% datos

In [ ]:

y_pred_probs = model_J.predict(valid_dataset_ignore)
y_true = df_valid_ignore[LABEL_COLUMS].values
y_pred = (y_pred_probs > umbrales(y_pred_probs)).astype(int)

mostrar_metricas(y_true,y_pred_probs, y_pred)

## Predicciones 

In [ ]:
# Selecciona un índice aleatorio del dataset de validación
import random


indice_aleatorio = random.randint(0, len(df_valid_ignore) - 1)
ruta_imagen = df_valid_ignore.iloc[indice_aleatorio]['Path']
etiqueta_real = df_valid_ignore.iloc[indice_aleatorio][LABEL_COLUMS].values

# Preprocesa la imagen igual que en validación
def cargar_y_preprocesar(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_with_pad(image, 150, 150)
    image = tf.cast(image, tf.float32) / 255.0
    return image

imagen_proc = cargar_y_preprocesar(ruta_imagen).numpy()
prediccion = model_J.predict(np.expand_dims(imagen_proc, axis=0))

# Usa los mejores umbrales calculados previamente
# Si no los tienes, usa 0.5 para todas las clases
try:
    umbrales_optimos = umbrales(prediccion)
except NameError:
    umbrales_optimos = [0.5] * len(LABEL_COLUMS)

prediccion_clase = (prediccion[0] > umbrales_optimos).astype(int)

# Muestra la imagen, la etiqueta real y la predicción
plt.imshow(imagen_proc)
plt.title(f"Etiqueta real: {etiqueta_real.astype(int)}\nPredicción: {prediccion_clase}")
plt.axis('off')
plt.show()

# Prueba 2 con pesos por clase


In [ ]:
#Creación de los dataSet  con enfoque U-Ignore
df_train_ignore = crear_dataFrame_UIgnore('/train.csv', 0.5)
train_dataset_ignore = obtener_dataset_Aug(df_train_ignore)

df_valid_ignore = crear_dataFrame_UIgnore('/valid.csv', 1.0)
valid_dataset_ignore = obtener_dataset_Aug(df_valid_ignore)

In [ ]:
model_J = crear_modelo_J()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    #Corta el entrenamiento automáticamente si el modelo deja de mejorar.
    EarlyStopping(monitor='val_auc', patience=4, mode='max', restore_best_weights=True),
    #Reduce automáticamente el learning rate si la métrica no mejora.
    #ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, mode='min')
    ReduceLROnPlateau(
    monitor='val_auc',     # Métrica a observar
    factor=0.5,            # Cuánto se reduce el learning rate (a la mitad en este caso)
    patience=2,            # Cuántas epochs esperar SIN mejora antes de reducir el LR
    mode='max',            # Porque queremos que el AUC crezca
    min_lr=1e-6,           # No baja más allá de este valor, máxima resolución del learing rate
    verbose=1              # Muestra en consola cuando se reduce
)
]
#Compilación:
model_J.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss= re_loss_P, metrics=[tf.keras.metrics.AUC(name='auc')])
model_J.fit(train_dataset_ignore, epochs=2, validation_data= valid_dataset_ignore, callbacks = callbacks)




## Métricas
Métricas para modelo_J con data Augmentation y pesos por clase enfoque U-ignore, 40% datos

In [ ]:
y_pred_probs = model_J.predict(valid_dataset_ignore)
y_true = df_valid_ignore[LABEL_COLUMS].values
y_pred = (y_pred_probs > umbrales(y_pred_probs)).astype(int)

mostrar_metricas(y_true,y_pred_probs, y_pred)

## Predicción del modelo

In [ ]:
# Mostrar una imagen de ejemplo con su predicción
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
import random

# Selecciona un índice aleatorio del dataset de validación
indice_aleatorio = random.randint(0, len(df_valid_ignore) - 1)
ruta_imagen = df_valid_ignore.iloc[indice_aleatorio]['Path']
etiqueta_real = df_valid_ignore.iloc[indice_aleatorio][LABEL_COLUMS].values

# Preprocesa la imagen igual que en validación
def cargar_y_preprocesar(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_with_pad(image, 150, 150)
    image = tf.cast(image, tf.float32) / 255.0
    return image

imagen_proc = cargar_y_preprocesar(ruta_imagen).numpy()
prediccion = model_J.predict(np.expand_dims(imagen_proc, axis=0))

# Usa los mejores umbrales calculados previamente 
# Si no los tienes, usa 0.5 para todas las clases
try:
    umbrales_optimos = umbrales(prediccion)
except NameError:
    umbrales_optimos = [0.5] * len(LABEL_COLUMS)

prediccion_clase = (prediccion[0] > umbrales_optimos).astype(int)

# Muestra la imagen, la etiqueta real y la predicción
plt.imshow(imagen_proc)
plt.title(f"Etiqueta real: {etiqueta_real.astype(int)}\nPredicción: {prediccion_clase}")
plt.axis('off')
plt.show()


# Prueba 3 Enfoque U-Zeroes

In [ ]:
df_train_zeroes = crear_dataFrame_UZeroes('train.csv',0.2)
train_dataset_zeroes = obtener_dataset(df_train_zeroes)

df_valid_zeroes = crear_dataFrame_UZeroes('valid.csv', 1.0)
valid_dataset_zeroes = obtener_dataset(df_valid_zeroes)


In [ ]:
model_J = crear_modelo_J()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
#Entrenamiento del modelo
callbacks = [
    #Corta el entrenamiento automáticamente si el modelo deja de mejorar.
    EarlyStopping(monitor='val_auc', patience=4, mode='max', restore_best_weights=True),
    #Reduce automáticamente el learning rate si la métrica no mejora.
    ReduceLROnPlateau(
    monitor='val_auc',     # Métrica a observar
    factor=0.5,            # Cuánto se reduce el learning rate (a la mitad en este caso)
    patience=2,            # Cuántas epochs esperar SIN mejora antes de reducir el LR
    mode='max',            # Porque queremos que el AUC crezca
    min_lr=1e-6,           # No baja más allá de este valor, máxima resolución del learing rate
    verbose=1              # Muestra en consola cuando se reduce
)
]
#Compilación:
model_J.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss= re_loss_, metrics=[tf.keras.metrics.AUC(name='auc')])
model_J.fit(train_dataset_zeroes, epochs=9, validation_data= valid_dataset_zeroes, callbacks = callbacks)

## Métricas

In [ ]:


y_pred_probs = model_J.predict(valid_dataset_zeroes)
y_true = df_valid_zeroes[LABEL_COLUMS].values
y_pred = (y_pred_probs > umbrales(y_pred_probs)).astype(int)

mostrar_metricas(y_true,y_pred_probs, y_pred)

# Prueba con Enfoque U-Zeroes balanceado

In [ ]:
df_train_zeroes = crear_dataFrame_UIgnore('train.csv',0.1)
df_bal = balancear_dataset(df_train_zeroes, LABEL_COLUMS)
train_dataset_zeroes = obtener_dataset(df_bal)

df_valid_zeroes = crear_dataFrame_UIgnore('valid.csv', 1.0)
valid_dataset_zeroes = obtener_dataset(df_valid_zeroes)

# CANTIDAD DE POSITIVOS POR CLASE

print(len(df_train_zeroes[LABEL_COLUMS]))
print('Cantidades antes de balancear:')
positivos_por_clase = (df_train_zeroes[LABEL_COLUMS]==1).sum().to_dict()
print("Cantidad de positivos por clase:")
for clase, cantidad in positivos_por_clase.items():
    print(f"{clase}: {cantidad}")

print()
print('Cantidades después de balancear:')
positivos_por_clase = (df_bal[LABEL_COLUMS]==1).sum().to_dict()
print("Cantidad de positivos por clase:")
for clase, cantidad in positivos_por_clase.items():
    print(f"{clase}: {cantidad}")

import matplotlib.pyplot as plt

for image, label in train_dataset_zeroes.take(1):
    plt.imshow(image[0].numpy())
    plt.title(f"Label: {label[0].numpy()}")
    plt.axis("off")
    plt.show()





In [ ]:
model_J = crear_modelo_J()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
#Entrenamiento del modelo
callbacks = [
    #Corta el entrenamiento automáticamente si el modelo deja de mejorar.
    EarlyStopping(monitor='val_auc', patience=4, mode='max', restore_best_weights=True),
    #Reduce automáticamente el learning rate si la métrica no mejora.
    ReduceLROnPlateau(
    monitor='val_auc',     # Métrica a observar
    factor=0.5,            # Cuánto se reduce el learning rate (a la mitad en este caso)
    patience=2,            # Cuántas epochs esperar SIN mejora antes de reducir el LR
    mode='max',            # Porque queremos que el AUC crezca
    min_lr=1e-6,           # No baja más allá de este valor, máxima resolución del learing rate
    verbose=1              # Muestra en consola cuando se reduce
)
]
#Compilación:
model_J.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss= re_loss_, metrics=[tf.keras.metrics.AUC(name='auc')])
model_J.fit(train_dataset_zeroes, epochs=9, validation_data= valid_dataset_zeroes, callbacks = callbacks)

## Métricas

In [ ]:
y_pred_probs = model_J.predict(valid_dataset_zeroes)
y_true = df_valid_zeroes[LABEL_COLUMS].values
y_pred = (y_pred_probs > umbrales(y_pred_probs)).astype(int)

mostrar_metricas(y_true,y_pred_probs, y_pred)

# Prueba DenseNet


In [ ]:
df_train_zeroes = crear_dataFrame_UZeroes('train.csv',0.05)
train_dataset_zeroes = obtener_dataset(df_train_zeroes)

df_valid_zeroes = crear_dataFrame_UZeroes('valid.csv', 1.0)
valid_dataset_zeroes = obtener_dataset(df_valid_zeroes)


In [ ]:
from tensorflow.keras import layers, Input, models
# Modificar tamaño de entrada
base_model = tf.keras.applications.DenseNet121(
    input_shape = (200,200,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = True

model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(5, activation='sigmoid')
])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
#Entrenamiento del modelo
callbacks = [
    #Corta el entrenamiento automáticamente si el modelo deja de mejorar.
    EarlyStopping(monitor='val_auc', patience=4, mode='max', restore_best_weights=True),
    #Reduce automáticamente el learning rate si la métrica no mejora.
    ReduceLROnPlateau(
    monitor='val_auc',     # Métrica a observar
    factor=0.5,            # Cuánto se reduce el learning rate (a la mitad en este caso)
    patience=2,            # Cuántas epochs esperar SIN mejora antes de reducir el LR
    mode='max',            # Porque queremos que el AUC crezca
    min_lr=1e-6,           # No baja más allá de este valor, máxima resolución del learing rate
    verbose=1              # Muestra en consola cuando se reduce
)
]
#Compilación:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss= re_loss_, metrics=[tf.keras.metrics.AUC(name='auc')])
model.fit(train_dataset_zeroes, epochs=9, validation_data= valid_dataset_zeroes, callbacks = callbacks)

## Métricas

In [ ]:
y_pred_probs = model.predict(valid_dataset_zeroes)
y_true = df_valid_zeroes[LABEL_COLUMS].values
y_pred = (y_pred_probs > umbrales(y_pred_probs,y_true)).astype(int)

mostrar_metricas(y_true,y_pred_probs, y_pred)

## Predicciones

In [ ]:
import random


indice_aleatorio = random.randint(0, len(df_valid_ignore) - 1)
ruta_imagen = df_valid_ignore.iloc[indice_aleatorio]['Path']
etiqueta_real = df_valid_ignore.iloc[indice_aleatorio][LABEL_COLUMS].values

# Preprocesa la imagen igual que en validación
def cargar_y_preprocesar(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_with_pad(image, 150, 150)
    image = tf.cast(image, tf.float32) / 255.0
    return image

imagen_proc = cargar_y_preprocesar(ruta_imagen).numpy()
prediccion = model_J.predict(np.expand_dims(imagen_proc, axis=0))

# Usa los mejores umbrales calculados previamente (por ejemplo, mejores_umbrales)
# Si no los tienes, usa 0.5 para todas las clases
try:
    umbrales_optimos = umbrales(prediccion)
except NameError:
    umbrales_optimos = [0.5] * len(LABEL_COLUMS)

prediccion_clase = (prediccion[0] > umbrales_optimos).astype(int)

# Muestra la imagen, la etiqueta real y la predicción
plt.imshow(imagen_proc)
plt.title(f"Etiqueta real: {etiqueta_real.astype(int)}\nPredicción: {prediccion_clase}")
plt.axis('off')
plt.show()